# DATAQUEST 

In [3]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

## Installing fastai==0.7.0

In [4]:
# !pip install fastai==0.7.0

## Importing libraries

In [5]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor , RandomForestClassifier
import re
import seaborn as sns

from sklearn import metrics
import io
import xgboost as xgb

## Getting the data in Google Colab

In [6]:
PATH = './data/'
!ls {PATH}

sample_submission.csv  test.csv  train.csv


In [7]:
train_raw = pd.read_csv(f'{PATH}train.csv',low_memory=False,parse_dates=['opened_at'])

In [8]:
test_raw = pd.read_csv(f'{PATH}test.csv',low_memory=False,parse_dates=['opened_at'])

## Setting some lists to measure progress 

In [9]:
train_map = []
test_map = []

##  Fuctions

In [10]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [11]:
def print_score(m):
    print(metrics.mean_absolute_error(m.predict(X_train),y_train),metrics.mean_absolute_error(m.predict(X_valid),y_valid))

In [12]:
def put_in_metrics(n):
    train_map.append(metrics.mean_absolute_error(n.predict(X_train),y_train))
    test_map.append(metrics.mean_absolute_error(n.predict(X_valid),y_valid))

In [13]:
def split_vals(a,n): return a[:n].copy() , a[n:].copy()

In [14]:
def extracting_num(df_raw):
  df = df_raw.copy()
  df['opened_by'] = df['opened_by'].str.split(expand=True)[2]
  df['location'] = df['location'].str.split(expand=True)[1]
  df['category'] = df['category'].str.split(expand=True)[1]
  df['subcategory'] = df['subcategory'].str.split(expand=True)[1]
  df['impact'] = df['impact'].str.split(expand=True)[0]
  df['urgency'] = df['urgency'].str.split(expand=True)[0]
  df['priority'] = df['priority'].str.split(expand=True)[0]
  return df
  

## EDA (DATA PREPROCESSING) 

In [15]:
train_raw.head().T

,0,1,2,3,4
Id,INC0000045,INC0000047,INC0000057,INC0000060,INC0000062
reassignment_count,0,0,0,0,0
reopen_count,0,0,0,0,0
update_count,0,0,0,0,0
made_sla,True,True,True,True,True
opened_by,Opened by 8,Opened by 397,Opened by 8,Opened by 180,Opened by 180
opened_at,2016-02-29 01:16:00,2016-02-29 04:40:00,2016-02-29 06:10:00,2016-02-29 06:38:00,2016-02-29 06:58:00
contact_type,Phone,Phone,Phone,Phone,Phone
location,Location 143,Location 165,Location 204,Location 204,Location 93
category,Category 55,Category 40,Category 20,Category 9,Category 53


In [16]:
train_raw = extracting_num(train_raw)

In [17]:
train_raw.T

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
Id,INC0000045,INC0000047,INC0000057,INC0000060,INC0000062,INC0000063,INC0000064,INC0000065,INC0000066,INC0000067,...,INC0028188,INC0028190,INC0028191,INC0028192,INC0028193,INC0028195,INC0028196,INC0028200,INC0028201,INC0028203
reassignment_count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
reopen_count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
update_count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
made_sla,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
opened_by,8,397,8,180,180,180,131,131,24,180,...,24,301,131,62,24,17,17,24,24,55
opened_at,2016-02-29 01:16:00,2016-02-29 04:40:00,2016-02-29 06:10:00,2016-02-29 06:38:00,2016-02-29 06:58:00,2016-02-29 07:08:00,2016-02-29 07:10:00,2016-02-29 07:38:00,2016-02-29 08:03:00,2016-02-29 08:03:00,...,2016-05-11 08:41:00,2016-05-11 08:41:00,2016-05-11 08:42:00,2016-05-11 08:44:00,2016-05-11 08:44:00,2016-05-11 08:45:00,2016-05-11 08:46:00,2016-05-11 08:50:00,2016-05-11 08:51:00,2016-05-11 08:52:00
contact_type,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,...,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone
location,143,165,204,204,93,93,143,108,161,143,...,161,56,161,143,108,204,93,97,51,204
category,55,40,20,9,53,20,53,44,55,9,...,9,37,42,53,20,42,37,42,53,26


In [18]:
display_all(train_raw.isna().sum().sort_index())

Id                        0
assigned_to            5409
category                 26
contact_type              0
impact                    0
knowledge                 0
location                 52
made_sla                  0
notify                    0
opened_at                 0
opened_by               714
priority                  0
reassignment_count        0
reopen_count              0
subcategory              56
target_days               0
update_count              0
urgency                   0
vendor                19994
dtype: int64

##### The attributes like impact , priority , urgency have similar kind of behaviour in general. We could test this out by making graph. The graph should be Id vs the attributes in ascending order(in order).

In [19]:
train_raw['priority'].value_counts()

3    18987
4      611
2      251
1      151
Name: priority, dtype: int64

In [20]:
train_raw['impact'].value_counts()

2    19174
3      599
1      227
Name: impact, dtype: int64

In [21]:
train_raw['urgency'].value_counts()

2    19117
3      557
1      326
Name: urgency, dtype: int64

##  Dropping columns (based on their interdependecy and not use)

In [22]:
column_names = ['Id','vendor']

In [23]:
train_raw.drop(columns='vendor',inplace=True)
train_raw.drop(columns='reassignment_count',inplace=True)
train_raw.drop(columns='reopen_count',inplace=True)
train_raw.drop(columns='update_count',inplace=True)

## Removing outliers

#### Outliers in update_count, opened_by

In [24]:
# train_raw.drop(train_raw[train_raw['update_count'] !=0].index,inplace=True)

In [25]:
# train_raw.update_count.value_counts()

####  The columns opened_by,location,category,subcategory can be made integer directly by stripping away the alphacharacters .  The columns impact,urgency,priority could be splitted.

In [26]:
add_datepart(train_raw,'opened_at')

In [27]:
for index,row in train_raw.iterrows() :
    if(row['target_days']>10):
        train_raw.drop(index,inplace=True)

## Splitting column data 

In [28]:
# train_raw['opened_by'] = train_raw['opened_by'].str.split(expand=True)[2]
#  train_raw.opened_by.astype('int')

In [29]:
# train_raw['opened_by'].isnull().sum()

In [30]:
# train_raw.dtypes

In [31]:
# train_raw.opened_by.astype('int',errors='ignore')

In [32]:
# train_raw['opened_by_na'] = train_raw['opened_by'].copy().isna()

In [33]:
# train_raw[train_raw['opened_by'].isna()] = 0 

In [34]:
# train_raw.opened_by_na.value_counts()

#### Now since we have many null values and we got to deal with that. We could do one hot encoding . Make a column of opened_by_na and then give 1 to value having NaN values and 0 to rest . And then in opened_by column we could make all of them as 0. Maybe this is all is pre implemented in a function in fastai . Got to check that. 

## Train_cats

In [35]:
train_cats(train_raw)

In [36]:
train_raw.impact.cat.set_categories(['3 - Low','2 - Medium','1 - High'],ordered=True,inplace=True)
train_raw.urgency.cat.set_categories(['3 - Low','2 - Medium','1 - High'],ordered=True,inplace=True)
train_raw.priority.cat.set_categories(train_raw.priority.cat.categories[::-1],ordered=True,inplace=True)

In [37]:
train_raw.isnull().sum().sort_index()/len(train_raw)

Id                           0.000000
assigned_to                  0.271087
category                     0.000906
contact_type                 0.000000
impact                       1.000000
knowledge                    0.000000
location                     0.001057
made_sla                     0.000000
notify                       0.000000
opened_atDay                 0.000000
opened_atDayofweek           0.000000
opened_atDayofyear           0.000000
opened_atElapsed             0.000000
opened_atIs_month_end        0.000000
opened_atIs_month_start      0.000000
opened_atIs_quarter_end      0.000000
opened_atIs_quarter_start    0.000000
opened_atIs_year_end         0.000000
opened_atIs_year_start       0.000000
opened_atMonth               0.000000
opened_atWeek                0.000000
opened_atYear                0.000000
opened_by                    0.029298
priority                     0.000000
subcategory                  0.001133
target_days                  0.000000
urgency     

# Preprocessing

In [38]:
def preprocessingl(df_raw):
    df = df_raw.copy()
    add_datepart(df,'opened_at')
    extracting_num(df)
    return df

In [39]:
train_raw['opened_atYear'].unique()

array([2016])

In [40]:
train_raw.drop(columns='opened_atYear',inplace=True)
train_raw.drop(columns='made_sla',inplace=True)
train_raw.drop(columns='impact',inplace=True)
train_raw.drop(columns='urgency',inplace=True)
train_raw.drop(columns='opened_atIs_year_start',inplace=True)

## Data after preprocessing 

In [41]:
display_all(train_raw.head().T)

,0,1,2,3,4
Id,INC0000045,INC0000047,INC0000057,INC0000060,INC0000062
opened_by,8,397,8,180,180
contact_type,Phone,Phone,Phone,Phone,Phone
location,143,165,204,204,93
category,55,40,20,9,53
subcategory,170,215,125,97,168
priority,3,3,3,3,3
assigned_to,NaN,Resolver 89,NaN,Resolver 125,NaN
knowledge,True,True,True,True,True
notify,Do Not Notify,Do Not Notify,Do Not Notify,Do Not Notify,Do Not Notify


# Preparing model

In [42]:
train , y, nas = proc_df(train_raw,'target_days')

In [43]:
train.T

,0,1,2,3,4,5,6,7,8,9,...,19987,19988,19990,19992,19993,19994,19995,19996,19997,19999
Id,1,2,3,4,5,6,7,8,9,10,...,13234,13235,13236,13237,13238,13239,13240,13241,13242,13243
opened_by,129,82,129,23,23,23,12,12,41,23,...,41,41,41,12,124,41,20,20,41,120
contact_type,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
location,31,42,76,76,169,169,31,5,40,31,...,40,129,40,40,31,5,76,169,172,76
category,35,24,5,43,33,5,33,28,35,43,...,10,10,43,26,33,5,26,21,26,10
subcategory,52,81,22,184,50,22,50,90,64,11,...,47,121,47,86,140,55,86,55,86,121
priority,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
assigned_to,0,148,0,15,0,0,143,80,0,96,...,50,50,99,67,113,150,19,146,50,0
knowledge,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
notify,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [44]:
from sklearn import preprocessing

In [45]:
label_encoder = preprocessing.LabelEncoder()

In [46]:
train['knowledge']= label_encoder.fit_transform(train['knowledge'])

In [47]:
train['opened_atIs_month_end']= label_encoder.fit_transform(train['opened_atIs_month_end'])

In [48]:
train['opened_atIs_month_start']= label_encoder.fit_transform(train['opened_atIs_month_start'])

In [49]:
train['opened_atIs_quarter_end']= label_encoder.fit_transform(train['opened_atIs_quarter_end'])

In [50]:
train['opened_atIs_quarter_start']= label_encoder.fit_transform(train['opened_atIs_quarter_start'])

In [51]:
train['opened_atIs_year_end']= label_encoder.fit_transform(train['opened_atIs_year_end'])

In [52]:
train.T

,0,1,2,3,4,5,6,7,8,9,...,19987,19988,19990,19992,19993,19994,19995,19996,19997,19999
Id,1,2,3,4,5,6,7,8,9,10,...,13234,13235,13236,13237,13238,13239,13240,13241,13242,13243
opened_by,129,82,129,23,23,23,12,12,41,23,...,41,41,41,12,124,41,20,20,41,120
contact_type,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
location,31,42,76,76,169,169,31,5,40,31,...,40,129,40,40,31,5,76,169,172,76
category,35,24,5,43,33,5,33,28,35,43,...,10,10,43,26,33,5,26,21,26,10
subcategory,52,81,22,184,50,22,50,90,64,11,...,47,121,47,86,140,55,86,55,86,121
priority,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
assigned_to,0,148,0,15,0,0,143,80,0,96,...,50,50,99,67,113,150,19,146,50,0
knowledge,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
notify,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [1299]:
n_valid = 3000 
n_trn = len(train) - n_valid
X_train , X_valid = split_vals(train,n_trn)
y_train , y_valid = split_vals(y,n_trn)

X_train.shape , X_valid.shape , y_train.shape , y_valid.shape

((10243, 21), (3000, 21), (10243,), (3000,))

In [1300]:
m = RandomForestClassifier(n_jobs=-1,random_state=3,min_samples_leaf=1)
m.fit(X_train,y_train)

/home/vaibhav/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=3, verbose=0,
                       warm_start=False)

In [1301]:
print_score(m)

0.053890461778775746 0.5523333333333333


In [1302]:
xg_reg = xgb.XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 50)

In [1303]:
xg_reg.fit(X_train,y_train)

XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=50, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [1304]:
print_score(xg_reg)

0.5989456213999804 0.5453333333333333


In [1305]:
from sklearn.neighbors import KNeighborsClassifier

In [1306]:
n = KNeighborsClassifier()

0.5709264863809431 0.5453333333333333


In [1308]:
import keras

In [1309]:
from keras.models import Sequential # intitialize the ANN
from keras.layers import Dense


In [1310]:
t = Sequential()

In [1311]:
t.add(Dense(units = 21, kernel_initializer = 'uniform', activation = 'relu', input_dim = 21))
t.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))
# t.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))
t.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'relu'))

In [1312]:
t.compile(optimizer = 'sgd', loss = 'mean_absolute_error', metrics = ['accuracy'])

In [1313]:
# t.fit(X_train, y_train, batch_size = 32, epochs = 200)

Epoch 1/200
10243/10243 [==============================] - 1s 74us/step - loss: 749.9665 - acc: 0.0025
Epoch 2/200
10243/10243 [==============================] - 1s 88us/step - loss: 5.9323 - acc: 0.0025
Epoch 3/200
10243/10243 [==============================] - 1s 88us/step - loss: 5.9323 - acc: 0.0025
Epoch 4/200
10243/10243 [==============================] - 1s 90us/step - loss: 5.9323 - acc: 0.0025
Epoch 5/200
10243/10243 [==============================] - 1s 89us/step - loss: 5.9323 - acc: 0.0025
Epoch 6/200
10243/10243 [==============================] - 1s 88us/step - loss: 5.9323 - acc: 0.0025
Epoch 7/200
10243/10243 [==============================] - 1s 88us/step - loss: 5.9323 - acc: 0.0025
Epoch 8/200
10243/10243 [==============================] - 1s 88us/step - loss: 5.9323 - acc: 0.0025
Epoch 9/200
10243/10243 [==============================] - 1s 88us/step - loss: 5.9323 - acc: 0.0025
Epoch 10/200
10243/10243 [==============================] - 1s 88us/step - loss: 5.9323 -

10243/10243 [==============================] - 1s 79us/step - loss: 5.9323 - acc: 0.0025
Epoch 161/200
10243/10243 [==============================] - 1s 79us/step - loss: 5.9323 - acc: 0.0025
Epoch 162/200
10243/10243 [==============================] - 1s 84us/step - loss: 5.9323 - acc: 0.0025
Epoch 163/200
10243/10243 [==============================] - 1s 79us/step - loss: 5.9323 - acc: 0.0025
Epoch 164/200
10243/10243 [==============================] - 1s 79us/step - loss: 5.9323 - acc: 0.0025
Epoch 165/200
10243/10243 [==============================] - 1s 79us/step - loss: 5.9323 - acc: 0.0025
Epoch 166/200
10243/10243 [==============================] - 1s 94us/step - loss: 5.9323 - acc: 0.0025
Epoch 167/200
10243/10243 [==============================] - 1s 89us/step - loss: 5.9323 - acc: 0.0025
Epoch 168/200
10243/10243 [==============================] - 1s 97us/step - loss: 5.9323 - acc: 0.0025
Epoch 169/200
10243/10243 [==============================] - 1s 94us/step - loss: 5.932

In [1314]:
# print_score(t)

5.93234403983208 5.514


# TEST 

In [1315]:
test_raw.T

,0,1,2,3,4,5,6,7,8,9,...,4908,4909,4910,4911,4912,4913,4914,4915,4916,4917
Id,INC0028205,INC0028208,INC0028209,INC0028210,INC0028211,INC0028212,INC0028213,INC0028214,INC0028215,INC0028216,...,INC0119457,INC0119983,INC0119987,INC0120268,INC0120303,INC0120304,INC0120319,INC0120495,INC0120835,INC0121064
reassignment_count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
reopen_count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
update_count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
made_sla,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
opened_by,Opened by 301,Opened by 501,Opened by 24,Opened by 24,Opened by 17,Opened by 24,Opened by 17,Opened by 17,Opened by 62,Opened by 501,...,Opened by 508,Opened by 508,Opened by 508,Opened by 508,Opened by 508,Opened by 8,Opened by 508,Opened by 14,Opened by 12,Opened by 12
opened_at,2016-05-11 08:52:00,2016-05-11 08:53:00,2016-05-11 08:55:00,2016-05-11 08:59:00,2016-05-11 08:59:00,2016-05-11 08:59:00,2016-05-11 09:00:00,2016-05-11 09:00:00,2016-05-11 09:01:00,2016-05-11 09:01:00,...,2017-02-13 11:31:00,2017-02-14 11:11:00,2017-02-14 11:15:00,2017-02-14 18:31:00,2017-02-15 01:52:00,2017-02-15 02:02:00,2017-02-15 07:09:00,2017-02-15 11:58:00,2017-02-16 09:09:00,2017-02-16 14:17:00
contact_type,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,...,IVR,IVR,IVR,IVR,IVR,Email,IVR,Phone,Email,Email
location,Location 161,Location 93,Location 161,Location 143,Location 204,Location 51,Location 204,Location 204,Location 55,Location 136,...,Location 246,Location 246,Location 246,Location 246,Location 246,Location 188,Location 246,Location 135,Location 204,Location 204
category,Category 9,Category 23,Category 32,Category 57,Category 57,Category 32,Category 23,Category 57,Category 42,Category 32,...,Category 50,Category 50,Category 50,Category 50,Category 50,Category 52,Category 50,Category 59,Category 42,Category 42


In [1316]:
Id = test_raw['Id']

In [1317]:
test_raw = preprocessingl(test_raw)

In [1318]:
apply_cats(test_raw,train_raw)

In [1319]:
test,_,_=proc_df(test_raw)

In [1320]:
test.shape , train.shape

((4918, 30), (13243, 21))

In [1321]:
test['priority'].unique()

array([0])

In [1322]:
test.drop(columns='vendor',inplace=True)
test.drop(columns='reassignment_count',inplace=True)
test.drop(columns='reopen_count',inplace=True)
test.drop(columns='update_count',inplace=True)
test.drop(columns='opened_atYear',inplace=True)

test.drop(columns='made_sla',inplace=True)
test.drop(columns='impact',inplace=True)
test.drop(columns='urgency',inplace=True)
test.drop(columns='opened_atIs_year_start',inplace=True)


In [1323]:

test['knowledge']= label_encoder.fit_transform(test['knowledge'])
test['opened_atIs_month_end']= label_encoder.fit_transform(test['opened_atIs_month_end'])
test['opened_atIs_month_start']= label_encoder.fit_transform(test['opened_atIs_month_start'])
test['opened_atIs_quarter_end']= label_encoder.fit_transform(test['opened_atIs_quarter_end'])
test['opened_atIs_quarter_start']= label_encoder.fit_transform(test['opened_atIs_quarter_start'])
test['opened_atIs_year_end']= label_encoder.fit_transform(test['opened_atIs_year_end'])


In [1332]:
 y_testt = xg_reg.predict(test)

In [1333]:
y_testt

array([5, 5, 5, ..., 5, 5, 5])

In [1334]:
dict = {
    'Id':Id,
    'target_days':y_testt
}

In [1335]:
submit = pd.DataFrame(dict)

In [1336]:
submit.to_csv('submit6.csv',header=True,index=False)